In [1]:
!pip install -U langchain-text-splitters
!pip install -U langchain-community langchain-text-splitters langchain
!pip install pypdf
!pip install -U langchain-huggingface
!pip install -qU langchain-chroma
!pip install -qU langchain
!pip install -qU langchain-groq
!pip install sentence-transformers
!pip install -q gradio

  Using cached huggingface_hub-0.36.2-py3-none-any.whl.metadata (15 kB)
Using cached huggingface_hub-0.36.2-py3-none-any.whl (566 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.4.1
    Uninstalling huggingface_hub-1.4.1:
      Successfully uninstalled huggingface_hub-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.
  Using cached huggingface_hub-1.4.1-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-1.4.1-py3-none-any.whl (553 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 0.36.2
    Uninstalling huggingface_hub-0.36.2:
      Successfully uninstalled huggingface_hub-0.36.2
ERROR: pip's dependency resolver does not currently take into ac

In [2]:
# DAY 1
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [3]:
loader = PyPDFLoader("../content/sample_data/assurance.pdf")
pdf = loader.load()

print(f"Document chargé : {len(pdf)} pages trouvées.")

Document chargé : 8 pages trouvées.


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,      # Taille cible de chaque morceau
    chunk_overlap = 150,    # Chevauchement
    separators = ["\n\n", "\n", " ", ""] # L'ordre de priorité pour couper
)

chunks = text_splitter.split_documents(pdf)

In [5]:
print(f"Nombre total de chunks pour {len(pdf)} pages : {len(chunks)}")

lengths = [len(c.page_content) for c in chunks]
import statistics
print(f"Taille moyenne : {statistics.mean(lengths)} caractères")

# Voir à quelle page appartient le chunk n°10
print(f"Le chunk 10 vient de la page : {chunks[10].metadata['page']}")

Nombre total de chunks pour 8 pages : 31
Taille moyenne : 803.7096774193549 caractères
Le chunk 10 vient de la page : 2


In [6]:
# DAY 2
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# Création de la base de données physique
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=hf_embeddings,
    persist_directory="./chroma_db"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
question = "Quelles sont les garanties de mon contrat ?"
question_embedding = hf_embeddings.embed_documents([question])

# Demande les 'k' meilleurs résultats
# Chroma fait l'embedding de la question et le calcul de similarité
docs = vectordb.similarity_search(question, k=1)

print(f"Question : {question}")
print("---")
print(f"Réponse: {docs[0].page_content}")

Question : Quelles sont les garanties de mon contrat ?
---
Réponse: Ce contrat est SANS TACITE reconduction, et à renouveler chaque année avant le 1er septembre.
Il garantit l'assuré en tant que locataire (ou occupant à titre gratuit), de Appartement situé(e) à l’adresse suivante :
2 route de Narbonne – N° App: 167  - Résidence: Résidence Bordegrande – Cplt adresse :  – Code postal et Ville : 31320 AUZEVILLE-
TOLOSANE 
Il couvre les risques suivants : 
- INCENDIE (sans franchise)
- DEGATS DES EAUX (sans franchise)
- PROTECTION JURIDIQUE
- INDEMNITE REDOUBLEMENT (garantie acquise uniquement pour les étudiants)
- INDIVIDUELLE ACCIDENT (garantie acquise uniquement pour les jeunes diplômés)
Pour toutes les questions relatives à la gestion de votre contrat, à vos déclarations de sinistre ou à vos besoins d’assistance, vous pouvez
contacter le n° 03 20 33 09 33.
La compagnie atteste avoir pris connaissance du fait que les personnes assurées au contrat seront amenées au sein du logement assur

In [10]:
# DAY 3
from langchain_groq import ChatGroq
import os

# os.environ["GROQ_API_KEY"] = "..."
os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=0.2
)

In [11]:
from langchain_classic.chains import RetrievalQA

# Transformer la base Chroma en "chercheur" (retriever)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Créer la chaîne RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # "stuff" veut dire : "donne tout le texte au LLM"
    retriever=retriever
)

In [12]:
response = qa_chain.invoke("Quelles sont les garanties de mon contrat ?")
print(response["result"])

Selon le contrat que vous avez fourni, les garanties de votre contrat sont les suivantes :

1. **INCENDIE** : sans franchise
2. **DEGATS DES EAUX** : sans franchise
3. **PROTECTION JURIDIQUE**
4. **INDEMNITE REDOUBLEMENT** : garantie acquise uniquement pour les étudiants
5. **INDIVIDUELLE ACCIDENT** : garantie acquise uniquement pour les jeunes diplômés

Il est également mentionné que le contrat couvre les dommages causés au matériel confié par l'employeur pour le télétravail, mais cela est considéré comme une garantie "dommages" acquise au contrat.


In [16]:
# DAY 4

In [19]:
import gradio as gr
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain

# 1. Configurer la mémoire
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [20]:
# 2. Créer la chaîne avec mémoire
qa_chat = ConversationalRetrievalChain.from_llm(
    llm=llm, # LLM Groq
    retriever=retriever, # retriever Chroma
    memory=memory
)

In [21]:
# 3. Fonction pour l'interface Gradio
def chat_interactif(message, history):
    response = qa_chat.invoke({"question": message})
    return response["answer"]

In [22]:
# 4. Lancer l'interface
demo = gr.ChatInterface(
    fn=chat_interactif,
    title="📚 Mon Assistant PDF Intelligent",
    description="Posez des questions sur votre document, je m'en souviendrai !",
    examples=["Fais-moi un résumé", "Quels sont les points clés ?"],
    theme="soft"
)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [23]:
demo.launch(share=True) # share=True crée un lien public de 72h !

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b239a59e544e3d561b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
